In [ ]:
#default_exp loss 

In [1]:
import os 
os.chdir('/content/drive/My Drive/Semi Supervised Learning/Self-Supervised-Learning-fastai2')

In [2]:
!sh initialise.sh

Requirement already up-to-date: fastcore==1.0.0 in /usr/local/lib/python3.6/dist-packages (1.0.0)


In [3]:
!sh nbdev_build.sh

  Using cached https://files.pythonhosted.org/packages/4a/c3/c6a5ba98213022a3363747ae88ea716809448f58cce46547fb90454416e7/fastcore-1.2.2-py3-none-any.whl
  Found existing installation: fastcore 1.0.0
    Uninstalling fastcore-1.0.0:
      Successfully uninstalled fastcore-1.0.0
Converted 01_learn.ipynb.
Converted 02_loss.ipynb.
Converted 03_models.ipynb.
Converted 04_utils.ipynb.
Converted 05_transforms.ipynb.
Converted index.ipynb.
  Using cached https://files.pythonhosted.org/packages/cc/92/233661d730b5613b4daf473cd28005bf2294fb1a858ce0bac57fbb7fa5ec/fastcore-1.0.0-py3-none-any.whl
ERROR: nbdev 1.1.4 has requirement fastcore>=1.2.0, but you'll have fastcore 1.0.0 which is incompatible.
  Found existing installation: fastcore 1.2.2
    Uninstalling fastcore-1.2.2:
      Successfully uninstalled fastcore-1.2.2


In [4]:
#export 
from ssl_fastai2.imports import *

In [5]:
#export 
class DotProduct(nn.Module):
  def forward(self, x, y):
    return torch.tensordot(x.unsqueeze(1), y.T.unsqueeze(0), dims = 2)
  
class NTXentLoss(nn.Module): # Normalised Temperature Scaled Cross Entropy
  def __init__(self, batch_size, similarity_type = 'cosine', temp = 0.1, use_softmax = False):
    super().__init__()
    self.use_softmax = use_softmax
    self.temp = temp

    if torch.cuda.is_available():
      self.device = torch.device('cuda')
    else:
      self.device = torch.device('cpu')

    self.batch_size = batch_size 
    self.neg_mask = self._get_neg_mask().to(self.device) 
    self.similarity = self._get_similarity(similarity_type.lower()) 
    self.criterion = nn.CrossEntropyLoss(reduction = 'sum')
    if use_softmax:
      self.softmax = nn.Softmax(dim = -1)

  def _get_similarity(self, sim_type):
    if sim_type == 'cosine': 
      return self.cosine_sim
    else:
      return DotProduct()

  def cosine_sim(self, x, y):
    return nn.CosineSimilarity(dim = -1)(x.unsqueeze(1), y.unsqueeze(0))

  @property 
  def _T_batch(self): return 2*self.batch_size

  def _get_neg_mask(self):
    diagonal = np.eye(self._T_batch)
    upper_diag = np.eye(self._T_batch, k = self.batch_size)
    lower_diag = np.eye(self._T_batch, k = -self.batch_size)
    mask = torch.from_numpy((diagonal + upper_diag + lower_diag))
    return (1 - mask).type(torch.bool)
  
  def forward(self, zi, zj):
    feature_rep = torch.cat([zi, zj], dim = 0)

    similarity_matrix = self.similarity(feature_rep, feature_rep)
    similarity_matrix = similarity_matrix/self.temp 

    negatives = similarity_matrix[self.neg_mask].view(self._T_batch, -1)

    l_pos = torch.diag(similarity_matrix, -self.batch_size)
    r_pos = torch.diag(similarity_matrix, self.batch_size) 
    positives = torch.cat([l_pos, r_pos]).view(self._T_batch, -1)

    labels = torch.zeros(self._T_batch).to(self.device).long()
    logits = torch.cat([positives, negatives], dim = 1)
    if self.use_softmax:
      logits = self.softmax(logits)
    loss = self.criterion(logits, labels)

    return loss/self._T_batch 

In [6]:
#export  
class BaseSSLLoss(nn.Module):
  def __init__(self, model, with_negatives = False, global_loss_func = None, branch_loss_func = None,  
               glob_loss_weight = 1., branch_loss_weight = 1.):
    super().__init__()      
    self.model = model
    self.with_negatives = with_negatives 
    self.glob_loss_weight = glob_loss_weight; self.branch_loss_weight = branch_loss_weight 
    self.global_loss_func = global_loss_func 
    if branch_loss_func:
      self.branch_loss_func = branch_loss_func 

In [7]:
#export 
class CGDLoss(BaseSSLLoss):
  def forward(self, *yb):
    '''Depending on Application targ could be positives or the augmentations of the same image '''

    if self.with_negatives:
      pred, targ, negatives, labels = yb 
    else:
      pred, targ, labels = yb

    targ = self.model(targ, glob = False)
    if self.with_negatives:
      neg = self.model(negatives, glob = False)

    global_loss = self.glob_loss_weight * self.global_loss_func(pred[0], labels)

    if self.with_negatives:
      target_loss = self.branch_loss_weight * self.branch_loss_func(pred[1], targ, neg)

    return global_loss + target_loss     

In [13]:
#export 
class BaseSIMClrLoss(BaseSSLLoss):
  def forward(self, pred, targ):
    targ = self.model(targ)
    loss = self.glob_loss_weight * self.global_loss_func(pred[0], targ[0])
    return loss 

def SIMClrLoss(model, batch_size, **kwargs):
  return BaseSIMClrLoss(model = model, global_loss_func = NTXentLoss(batch_size, **kwargs))

In [14]:
from ssl_fastai2.models import SIMClrModel 
zi, zj = [torch.rand((64, 256))], torch.rand((64, 3, 224, 224))
SIMClrLoss(SIMClrModel(create_body(resnet34), 256), 64)(zi, zj)

tensor(12.9622, grad_fn=<MulBackward0>)